# Train NN by Iris Dataset using 10-fold cross validation

## Imports

In [1]:
import tensorflow as tf
import numpy as np
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

### Reset graph

In [2]:
tf.reset_default_graph()

### Load raw data

In [3]:
raw_data = load_iris()

### Split Train-Test Data

In [4]:
X_train, X_test, Y_train, Y_test = train_test_split(
    raw_data.data, raw_data.target,
    test_size=1 / 3, random_state=42, stratify=raw_data.target)


### Scalling

In [5]:
X_scaller = MinMaxScaler(feature_range=(0, 1))

### Pre-process

In [6]:
X_train_scaled = X_scaller.fit_transform(X_train)
X_test_scaled = X_scaller.fit_transform(X_test)

In [7]:
one_hot_encoder = OneHotEncoder(sparse=False, categories='auto')
Y_train_encoded = one_hot_encoder.fit_transform(Y_train.reshape((-1, 1)))
Y_test_encoded = one_hot_encoder.fit_transform(Y_test.reshape((-1, 1)))

## Model Definition

### Define model parameters

In [8]:
learning_rate = 0.01
training_epochs = 10000

### Define layer sizes

In [9]:
layer_1_size = 200
layer_2_size = 100

### Define number of inputs and outputs

In [10]:
num_input = X_train_scaled.shape[1]
num_output = len(np.unique(Y_train, axis=0))

### Define layers

In [11]:
with tf.variable_scope("input"):
    X = tf.placeholder(tf.float32, shape=[None, num_input])

#### Layer 1

In [12]:
with tf.variable_scope("layer_1"):
    weights = tf.get_variable(
        "weights1", 
        shape=[num_input, layer_1_size],
        initializer=tf.contrib.layers.xavier_initializer())
    biases = tf.get_variable(
        "bias1",
        shape=[layer_1_size],
        initializer=tf.zeros_initializer()
    )
    layer_1_output = tf.nn.relu(tf.matmul(X, weights) + biases)

#### Layer 2

In [13]:
with tf.variable_scope("layer_2"):
    weights = tf.get_variable(
        "weights2", 
        shape=[layer_1_size, layer_2_size],
        initializer=tf.contrib.layers.xavier_initializer())
    biases = tf.get_variable(
        "bias2",
        shape=[layer_2_size],
        initializer=tf.zeros_initializer()
    )
    layer_2_output = tf.nn.relu(tf.matmul(layer_1_output, weights) + biases)

#### Output

In [14]:
with tf.variable_scope("output"):
    weights = tf.get_variable(
        "weights3", 
        shape=[layer_2_size, num_output],
        initializer=tf.contrib.layers.xavier_initializer())
    biases = tf.get_variable(
        "bias3",
        shape=[num_output],
        initializer=tf.zeros_initializer()
    )
    prediction = tf.matmul(layer_2_output, weights) + biases

#### Cost

In [15]:
with tf.variable_scope("cost"):
    Y = tf.placeholder(tf.float32, shape=[None, num_output])
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=prediction))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



#### Optimize

In [20]:
with tf.variable_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

#### Accuracy

In [17]:
with tf.variable_scope("accuracy"):
    corr_pred = tf.equal(tf.argmax(Y, axis=1), tf.argmax(prediction, axis=1))
    accuracy = tf.reduce_mean(tf.cast(corr_pred, tf.float32))

#### Log Result

In [18]:
with tf.variable_scope("logging"):
    tf.summary.scalar("current_cost", cost)
    tf.summary.scalar("current_accuracy", accuracy)
    summary = tf.summary.merge_all()

## Running Session

In [25]:
with tf.Session() as session:
    session.run(tf.global_variables_initializer())

    train_writer = tf.summary.FileWriter('assignments/neural-network/tmp/train')
    test_writer = tf.summary.FileWriter('assignments/neural-network/tmp/test')

    train_feed = {X: X_train_scaled, Y: Y_train_encoded}
    test_feed = {X: X_test_scaled, Y: Y_test_encoded}
    
    for epoch in range(training_epochs):
        session.run(optimizer, feed_dict=train_feed)

        if epoch % 100 == 0:
            train_cost, train_summary = session.run(
                [cost, summary], feed_dict=train_feed
            )
            test_cost, test_summary = session.run(
                [cost, summary], feed_dict=test_feed
            )

            train_acc = session.run(accuracy, feed_dict=train_feed)
            test_acc = session.run(accuracy, feed_dict=test_feed)

            print(epoch, train_cost, test_cost, train_acc, test_acc)

            train_writer.add_summary(train_summary, epoch)
            test_writer.add_summary(test_summary, epoch)

    print("Training is completed", "\n")
    final_train_acc = session.run(accuracy, feed_dict=train_feed)
    final_test_acc = session.run(accuracy, feed_dict=test_feed)
    
    print("Final Train Accuracy:", final_train_acc)
    print("Final Test Accuracy:", final_test_acc)
    
    train_writer.close()
    test_writer.close()

0 1.1090431 1.1084955 0.33 0.34
100 0.95462245 0.95730865 0.67 0.66


200 0.8345694 0.84017473 0.67 0.68


300 0.71783996 0.7260194 0.69 0.7
400 0.6141985 0.62493557 0.74 0.74


500 0.53154075 0.5444565 0.82 0.8
600 0.46924576 0.4838156 0.86 0.84


700 0.42051604 0.4360463 0.91 0.9
800 0.38131768 0.39751828 0.92 0.96


...

9100 0.029967535 0.065796845 1.0 0.96
9200 0.029721308 0.0658021 1.0 0.96


9300 0.029479437 0.06581159 1.0 0.96
9400 0.029241845 0.06582486 1.0 0.96


9500 0.029008323 0.06583765 1.0 0.96
9600 0.028778816 0.06585192 1.0 0.96


9700 0.028553095 0.065869056 1.0 0.96
9800 0.028331036 0.06588884 1.0 0.96


9900 0.028112877 0.06591042 1.0 0.96
Training is completed 

Final Train Accuracy: 1.0
Final Test Accuracy: 0.96
